In [72]:
import HierPortfolios as hp
import pandas as pd
import numpy as np
import dateutil as du

In [73]:
df = pd.read_excel("C:\\Users\\felip\\OneDrive\\Área de Trabalho\\BDAQ\\data\\ibrx_mensal.xlsx", na_values="-")

In [74]:
# replace column names
string_to_remove = "Retorno\ndo fechamento\nem 1 mês\nEm moeda orig\najust p/ prov\n"
df.columns = [col.replace(string_to_remove, "") for col in df.columns]
df

,Data,RRRP3,ALPA4,ABEV3,AMER3,ASAI3,AZUL4,B3SA3,BIDI11,BPAN4,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VIIA3,VBBR3,WEGE3,YDUQ3,IBOV
0,Dez-1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jan-1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000
2,Fev-1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.953271
3,Mar-1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.943089
4,Abr-1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.388060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,Mar-2022,23.524203,1.321415,1.315789,7.049180,21.025260,-5.417161,9.079487,5.267365,5.794206,...,1.912823,10.075643,-3.295173,-5.030591,7.580624,10.026385,-0.890208,18.738209,-1.458824,6.060785
652,Abr-2022,10.465950,-24.817798,-5.714286,-26.493109,-5.438771,-8.152174,-15.340547,-28.443396,-11.614731,...,-2.248006,-12.122045,-7.632509,-14.669134,-12.876569,-28.776978,-9.177823,-13.679381,-22.436002,-10.102623
653,Mai-2022,5.451006,10.969388,-2.272727,-16.250000,3.664921,-8.466090,-3.984962,-17.007251,-14.469798,...,6.824926,-11.260557,13.361958,-3.094607,3.505823,5.723906,-7.954545,-15.714286,0.372208,3.220684
654,Jun-2022,-27.897436,-12.000000,-5.567301,-33.184080,-9.911616,-38.438588,-13.166237,-17.394758,-23.433584,...,-10.593584,-18.011984,-14.335421,-21.076642,-11.193597,-38.853503,-14.043210,4.427862,-19.221261,-11.502920


In [ ]:
df = df.dropna(thresh=180, axis=1)
df
